In [36]:
#NLP FINAL PROJECT
#ANGIE GEORGARAS, PAULINA ADAMNSKI, DANIELLA POMBO, LINETTE MALIAKAL, ALEX ROSE, JACK BROOKS
import pandas as pd
import numpy as np
from keras.preprocessing import *
import re
from keras.models import Sequential
from keras import layers
import nltk
from nltk.stem.porter import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 41, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/usr/local/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/usr/local/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: dlopen(/usr/local/lib/python3.6/site-packages/tenso

In [2]:
GenX = pd.read_csv('./data/original/GenX_sub_data.csv')
Teen = pd.read_csv('./data/original/teenagers_sub_data.csv')

In [3]:
#Removes bots, comments with links, and makes all comments lowercase
def clean_data(data):
    if (data['User'] != 'AutoModerator'):
        data['Comment'] = data['Comment'].lower()
        if('https://' not in data['Comment']):
            return data
    else:
            data = data.drop('User')      

In [4]:
GenX = GenX.apply(clean_data,axis=1)
Teen = Teen.apply(clean_data,axis=1)

In [5]:
#drop deleted comments and unnecesssary columns
GenX = GenX.dropna()
Teen = Teen.dropna()

GenX = GenX.drop(['Unnamed: 0','SubredditofOrgin', 'Submission', 'User'], axis=1)
Teen = Teen.drop(['Unnamed: 0','SubredditofOrgin', 'Submission', 'User'], axis=1)

In [6]:
GenX.to_csv('./data/genx_processed.csv')
Teen.to_csv('./data/teen_processed.csv')

In [7]:
def import_txtF():
    #Import data sets and assign corresponding labels
        #'teen_processed.csv' - reddit teen text
        #'genx_processed.csv' - reddit GenX text
    
    teen = pd.read_csv('./data/teen_processed.csv', header = [0])
    tn_label = np.zeros((teen.shape[0], 1), dtype=int)
    gX = pd.read_csv('./data/genx_processed.csv', header = [0])
    gX_label = np.ones((gX.shape[0], 1), dtype=int)

    return teen, tn_label, gX, gX_label

In [19]:
def rdttxt_processing():
    tn, tn_l, genX, genX_l = import_txtF()
    
    #Create dataset (combining all data into one data set of text and labels)
    dat = pd.concat([tn, genX])
    dat = dat.drop(dat.columns[[0]], axis = 1)
    dat['labels'] = np.concatenate([tn_l, genX_l])
    
    #Clean data set of unwanted characters
    dat.replace(to_replace = r'[0-9"\*><\',]', value = ' ', regex = True, inplace = True)       
    
    #Shuffle data set
    dat = dat.sample(frac=1).reset_index(drop=True)
    
    doc = dat.iloc[:,0]
    doc = vec(doc) #Vectorize text data
    train_txt, test_txt, train_label, test_label = train_test_split(doc, dat['labels'], test_size= .2, train_size = .8, shuffle = True)
    
    return train_txt, test_txt, train_label, test_label

In [20]:
def vec(trn, tst= None):
    # Code inspired by https://stackoverflow.com/questions/36182502/add-stemming-support-to-countvectorizer-sklearn/36191362
    
    analyzer = TfidfVectorizer().build_analyzer()
    
    #Implements lemmalization funcationality to Vectorizer (tokenization & bag-of-words) method
    def stems(doc):
        ps = PorterStemmer() 
        return (ps.stem(w) for w in analyzer(doc))
    
    #remove stop words (ANGIE)
    
    #Intialize vectorizer
    vectorize = TfidfVectorizer(stop_words = 'english', max_df = 1, min_df = 1, analyzer = stems)
    
    vec_trn = vectorize.fit_transform(trn) #Fits and Transforms training data
    vocab = vectorize.get_feature_names() #identifies the text (words) associated w/ each index value

    #return vec_trn, vec_tst
    return vec_trn

In [21]:
#Naive Bayes Classifier
def NB():
    nbs = ComplementNB()
    nbs.fit(train_txt, train_label)
    
    print("naive bayes")

In [22]:
#Logistic Regression Classifier
def LR():
    lr = LogisticRegression()
    lr.fit(train_txt, train_label)
    
    print("logistic regression")

In [34]:
#CNN Classifier
def CNN():
    embedding_dim = 100

    #create architecture
    cnn = Sequential()
    cnn.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    cnn.add(layers.Conv1D(128, 5, activation='relu'))
    cnn.add(layers.GlobalMaxPooling1D())
    cnn.add(layers.Dense(10, activation='relu'))
    cnn.add(layers.Dense(1, activation='sigmoid'))
    cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()

    history = cnn.fit(train_txt, train_label,
                        epochs=10,
                        verbose=False,
                        validation_data=(test_txt, test_label),  batch_size=10)
    loss, accuracy = model.evaluate(train_txt,train_label, verbose=False)
    print("Training Accuracy: {:.4f}".format(accuracy))
    loss, accuracy = model.evaluate(test_txt, test_label, verbose=False)
    print("Testing Accuracy:  {:.4f}".format(accuracy))
    plot_history(history)

In [24]:
#LSTM Classifier (ANGIE)

In [25]:
#Model Evaluation (GRIDSEARCH = ANGIE)
def model_evaluation():

    print('')

In [26]:
#Model Training/Tuning
def model_training():
    
    print('')

In [35]:
def main():
    rdttxt_processing()
    CNN()
    #Wild out B
   
main()

NameError: name 'Sequential' is not defined